# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [79]:
# Date: 04.01.25
# Note: Not using the openai approach, but...
# Big success in getting Flash to make function calls. 
# Then copying idea from day 3 was able to use GPT to get the Flash function calling woking with Gradio!!!
# Then in hacky way improved above, so it doesn't call the 'get_price_function' every time it sees a city,
# it now also looks for related key words (ticket, cost, etc.), so if the question is unrelated to travel it can 
# answer without spitting out ticket price.
# I will highlight that code with # -- key code -- 

In [99]:
# imports

import os
import json
from dotenv import load_dotenv
#from openai import OpenAI
import gradio as gr

In [100]:
# # Load environment variables in a file called .env

# load_dotenv()
# api_key = os.getenv('OPENAI_API_KEY')

# # Check the key

# if not api_key:
#     print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
# #elif not api_key.startswith("sk-proj-"):
# elif not api_key.startswith("A*"):    
#     print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
# elif api_key.strip() != api_key:
#     print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
# else:
#     print("API key found and looks good so far!")

# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
#elif not api_key.startswith("sk-proj-"):
elif not api_key.startswith("A*"):    
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")



An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook


In [101]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [102]:
# ---------- Key code ----------

In [103]:
# Ignore provided code as not using open ai
# -- Was able to use Flash and make function calls using the theme of this notebook.
# Do NOT edit

# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

# def chat(message, history):
#     messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
#     response = openai.chat.completions.create(model=MODEL, messages=messages)
#     return response.choices[0].message.content

# gr.ChatInterface(fn=chat, type="messages").launch()


# Replacing above with Gem
import os
import json
import google.generativeai as genai

# Configure the API key
genai.configure(api_key=api_key)
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash",
                              generation_config=generation_config,)

# Path to save the history file
history_file = "conversation_history.json"

# Function to load history from file
def load_history():
    if os.path.exists(history_file):
        with open(history_file, "r") as f:
            return json.load(f)
    else:
        return []

# Function to save history to file
def save_history(history):
    with open(history_file, "w") as f:
        json.dump(history, f)


# --- Your ticket price function (Tool) ---
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}
def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

# Function to handle a chat with the Generative AI model
def ask_chat_model(prompt, history):
    chat_session = model.start_chat(history=history)
    response = chat_session.send_message(prompt)
    # Add the model's response to the history
    history.append({
        "role": "model",
        "parts": [response.text]
    })
    # Save the updated history to file
    save_history(history)
    return response.text

# Function to check if a question is about ticket prices and respond accordingly
def handle_ticket_price_request(prompt):
    # Check if the prompt is related to a ticket price request (contains a city)
    cities = list(ticket_prices.keys())
    for city in cities:
        if city.lower() in prompt.lower():
            # If it's a ticket price request, call the tool function
            price = get_ticket_price(city)
            return f"The ticket price for {city.capitalize()} is {price}."
    return None

# Main function to ask a question and determine if it's a ticket request or a chat request
def ask_question(prompt, history):
    # First, check if the prompt is related to a ticket price
    ticket_response = handle_ticket_price_request(prompt)
    if ticket_response:
        # If it's a ticket price request, return the response
        return ticket_response
    # Otherwise, proceed with the chat model if it's not about a ticket price
    history.append({
        "role": "user",
        "parts": [prompt]
    })
    return ask_chat_model(prompt, history)

# Example usage:
history = load_history()  # Load previous conversation history if it exists
prompt = "How much is the ticket to London?"
#prompt = "How much is the ticket to Spain?"
#prompt = "Is boxing effective in a street fight?"
#prompt = "What about karate?"
#prompt = "How much is a trip to Paris?"
response = ask_question(prompt, history)
print(response)


Tool get_ticket_price called for london
The ticket price for London is $799.


In [104]:
# Adds above capability to Gradio! (Could try to make this code more consistent with day 3 if desired, but this works)

In [105]:
# Copy of above (i.e. Flash with function call, but also adds Gradio UI)
# -- Do NOT edit ---
import os
import json
import google.generativeai as genai
import gradio as gr

# Configure the API key
genai.configure(api_key=api_key)
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash", generation_config=generation_config)

# Path to save the history file
history_file = "conversation_history.json"

# Function to load history from file
def load_history():
    if os.path.exists(history_file):
        with open(history_file, "r") as f:
            return json.load(f)
    else:
        return []

# Function to save history to file
def save_history(history):
    with open(history_file, "w") as f:
        json.dump(history, f)

# --- Your ticket price function (Tool) ---
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

# Function to handle a chat with the Generative AI model
def ask_chat_model(prompt, history):
    chat_session = model.start_chat(history=history)
    response = chat_session.send_message(prompt)
    # Add the model's response to the history
    history.append({
        "role": "model",
        "parts": [response.text]
    })
    # Save the updated history to file
    save_history(history)
    return response.text

# Function to check if a question is about ticket prices and respond accordingly
def handle_ticket_price_request(prompt):
    # Check if the prompt is related to a ticket price request (contains a city)
    cities = list(ticket_prices.keys())
    for city in cities:
        if city.lower() in prompt.lower():
            # If it's a ticket price request, call the tool function
            price = get_ticket_price(city)
            return f"The ticket price for {city.capitalize()} is {price}."
    return None

# Main function to ask a question and determine if it's a ticket request or a chat request
def ask_question(prompt, history):
    # First, check if the prompt is related to a ticket price
    ticket_response = handle_ticket_price_request(prompt)
    if ticket_response:
        # If it's a ticket price request, return the response
        return ticket_response #, history
    # Otherwise, proceed with the chat model if it's not about a ticket price
    history.append({
        "role": "user",
        "parts": [prompt]
    })
    response = ask_chat_model(prompt, history)
    #return response, history
    return response

# Function to handle the Gradio interface
def gradio_chat(message, history):
    # Process the message, handling both ticket and chat requests
    response, updated_history = ask_question(message, history)
    #return response, updated_history
    return response

# Create the Gradio interface
def chat_with_llm(prompt):
    # Load conversation history
    history = load_history()
    # Get response from the model
    response = ask_question(prompt, history)
    return response

# Create a Gradio Interface with a text input and output
iface = gr.Interface(
    fn=chat_with_llm,          # Function to be called
    inputs=gr.Textbox(label="Ask a Question", placeholder="Type your question here..."), # Text input for user prompt
    outputs=gr.Textbox(label="Response", interactive=True), # Text output for model's response
    title="Gemini LLM Chat",    # Title of the interface
    description="Ask questions and converse with the Gemini LLM model."  # Description of the interface
)

# Launch the Gradio interface
iface.launch()


* Running on local URL:  http://127.0.0.1:7920

To create a public link, set `share=True` in `launch()`.


In [75]:
# The above uses key word to trigger the function, i.e. if text contains 'london' then trigger ticket function. But what if customer is asking
# how to get to london, not whats the ticket price. Below tries to fix that.

In [106]:
# Very hacky. It now looks for the city and key words related to travel.
# i.e. if it sees a city, it then looks for travel related words also, before calling function.

import os
import json
import google.generativeai as genai
import gradio as gr

# Configure the API key
genai.configure(api_key=api_key)
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash", generation_config=generation_config)

# Path to save the history file
history_file = "conversation_history.json"

# Function to load history from file
def load_history():
    if os.path.exists(history_file):
        with open(history_file, "r") as f:
            return json.load(f)
    else:
        return []

# Function to save history to file
def save_history(history):
    with open(history_file, "w") as f:
        json.dump(history, f)

# --- Your ticket price function (Tool) ---
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

# Function to check if the question is related to ticket prices (based on keywords)
def is_ticket_price_request(prompt):
    # List of cities to check for ticket price mention
    cities = list(ticket_prices.keys())
    
    # Check if the prompt contains a city and a price-related question
    for city in cities:
        if city.lower() in prompt.lower():
            # Further check if the user is asking for ticket prices (avoid questions like "how to get to")
            if "ticket" in prompt.lower() or "price" in prompt.lower() \
            or "travel" in prompt.lower() or "cost" in prompt.lower():
                return True
    return False
 
# Function to handle a chat with the Generative AI model
def ask_chat_model(prompt, history):
    chat_session = model.start_chat(history=history)
    response = chat_session.send_message(prompt)
    # Add the model's response to the history
    history.append({
        "role": "model",
        "parts": [response.text]
    })
    # Save the updated history to file
    save_history(history)
    return response.text

# Main function to ask a question and determine if it's a ticket request or a chat request
def ask_question(prompt, history):
    # Check if the prompt is related to a ticket price request
    if is_ticket_price_request(prompt):
        # Extract the city name from the question if it's a ticket price request
        cities = list(ticket_prices.keys())
        for city in cities:
            if city.lower() in prompt.lower():
                # If it's a ticket price request, call the tool function
                price = get_ticket_price(city)
                return f"The ticket price for {city.capitalize()} is {price}."
    
    # If it's not a ticket price request, proceed with the chat model
    history.append({
        "role": "user",
        "parts": [prompt]
    })
    return ask_chat_model(prompt, history)

# Function to handle the Gradio interface
def gradio_chat(message, history):
    # Process the message, handling both ticket and chat requests
    response = ask_question(message, history)
    return response

# Create the Gradio interface
def chat_with_llm(prompt):
    # Load conversation history
    history = load_history()
    # Get response from the model
    response = ask_question(prompt, history)
    return response

# Create a Gradio Interface with a text input and output
iface = gr.Interface(
    fn=chat_with_llm,          # Function to be called
    inputs=gr.Textbox(label="Ask a Question", placeholder="Type your question here..."), # Text input for user prompt
    outputs=gr.Textbox(label="Response", interactive=True), # Text output for model's response
    title="Gemini LLM Chat",    # Title of the interface
    description="Ask questions and converse with the Gemini LLM model."  # Description of the interface
)

# Launch the Gradio interface
iface.launch()


* Running on local URL:  http://127.0.0.1:7921

To create a public link, set `share=True` in `launch()`.


In [77]:
# ---------- Key code ----------

In [119]:
# Copy of above - but trying swap out key word search for llm call when checking if question relates to travel
# Very hacky. It now looks for the city and key words related to travel.
# i.e. if it sees a city, it then looks for travel related words also, before calling function.

import os
import json
import google.generativeai as genai
import gradio as gr

# Configure the API key
genai.configure(api_key=api_key)
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash", generation_config=generation_config)

# Path to save the history file
history_file = "conversation_history.json"

# Function to load history from file
def load_history():
    if os.path.exists(history_file):
        with open(history_file, "r") as f:
            return json.load(f)
    else:
        return []

# Function to save history to file
def save_history(history):
    with open(history_file, "w") as f:
        json.dump(history, f)

# --- Your ticket price function (Tool) ---
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")


# ---------- below is the edit ----------

# # Function to check if the question is related to ticket prices (based on keywords)
def is_ticket_price_request(prompt):
    # List of cities to check for ticket price mention
    cities = list(ticket_prices.keys())
    
    # Check if the prompt contains a city and a price-related question
    for city in cities:
        if city.lower() in prompt.lower():
            # Further check if the user is asking for ticket prices (avoid questions like "how to get to")
            if "ticket" in prompt.lower() or "price" in prompt.lower() \
            or "travel" in prompt.lower() or "cost" in prompt.lower():
                return True
    return False

# Function to check if the question is related to ticket prices (based on llm)
def is_ticket_price_request(prompt):
    # List of cities to check for ticket price mention
    cities = list(ticket_prices.keys())
    system = f"""Please review {prompt} to check if it is specifically ticket related, i.e 
            Question includes things like - how much, cost, price, ticket, etc. then respond with 'y' 
            if {prompt} is general in nature, i.e
            Question includes things like - is it a nice place, how to get to, where is then respond with 'n'"""
    chat_session = model.start_chat(history=[  ])
    response = chat_session.send_message(system)
    response.text
    for city in cities:
        if city.lower() in prompt.lower():
            if 'y\n' in response.text:
                # Do something if 'n\n' is found in the string
                print("Found 'y\\n' in the string, so is price related!")
                print('prompt:', prompt)
                return True
    return False

    # system = f"""if {prompt} is specifically related ticket price, i.e 
    #         Question relates to price, ticket, cost, etc.; respond with 'y' 
    #         if {prompt} is general in nature, i.e
    #         Question relates to, how do i get to, where is, etc; respond with 'n'"""


# Function to check if the question is related to ticket prices (based on keywords)
# def is_ticket_price_request(prompt):
#     system = f"""if {prompt} is related ticket price, for example when a customer asks 'How much is a ticket to this city'; respond with 'y' else respond with 'n'"""
#     chat_session = model.start_chat(history=[  ])
#     response = chat_session.send_message(system)
#     response.text

#     if 'n\n' in response.text:
#     # Do something if 'n\n' is found in the string
#          print("Found 'n\\n' in the string, so not price related!")
#          return True
#     # else:
#     # # Do something if 'n\n' is not found
#     #      print("Did not find 'y\\n' in the string, so NOT price related.")
#     #      return False

# ---------- above is the edit ----------


 
# Function to handle a chat with the Generative AI model
def ask_chat_model(prompt, history):
    chat_session = model.start_chat(history=history)
    response = chat_session.send_message(prompt)
    # Add the model's response to the history
    history.append({
        "role": "model",
        "parts": [response.text]
    })
    # Save the updated history to file
    save_history(history)
    return response.text

# Main function to ask a question and determine if it's a ticket request or a chat request
def ask_question(prompt, history):
    # Check if the prompt is related to a ticket price request
    if is_ticket_price_request(prompt):
        # Extract the city name from the question if it's a ticket price request
        cities = list(ticket_prices.keys())
        for city in cities:
            if city.lower() in prompt.lower():
                # If it's a ticket price request, call the tool function
                price = get_ticket_price(city)
                return f"The ticket price for {city.capitalize()} is {price}."
    
    # If it's not a ticket price request, proceed with the chat model
    history.append({
        "role": "user",
        "parts": [prompt]
    })
    return ask_chat_model(prompt, history)

# Function to handle the Gradio interface
def gradio_chat(message, history):
    # Process the message, handling both ticket and chat requests
    response = ask_question(message, history)
    return response

# Create the Gradio interface
def chat_with_llm(prompt):
    # Load conversation history
    history = load_history()
    # Get response from the model
    response = ask_question(prompt, history)
    return response

# Create a Gradio Interface with a text input and output
iface = gr.Interface(
    fn=chat_with_llm,          # Function to be called
    inputs=gr.Textbox(label="Ask a Question", placeholder="Type your question here..."), # Text input for user prompt
    outputs=gr.Textbox(label="Response", interactive=True), # Text output for model's response
    title="Gemini LLM Chat",    # Title of the interface
    description="Ask questions and converse with the Gemini LLM model."  # Description of the interface
)

# Launch the Gradio interface
iface.launch()


* Running on local URL:  http://127.0.0.1:7929

To create a public link, set `share=True` in `launch()`.


Found 'y\n' in the string, so is price related!
prompt: How much to get to Berlin?
Tool get_ticket_price called for berlin


In [115]:
#prompt = "how much for a ticket to london?"
#prompt = "What to kittens eat"
prompt = "how do i get to Paris?"
system = f"""if {prompt} is specifically related ticket price, i.e 
            if a customer asks 'How much is a ticket to this city'; respond with 'y' 
            if the {prompt} is general in nature, i.e
            'how do i get to this city; respond with 'n'"""
chat_session = model.start_chat(history=[  ])
response = chat_session.send_message(system)
response.text

if 'n\n' in response.text:
    # Do something if 'n\n' is found in the string
    print("Found 'n\\n' in the string!")
else:
    # Do something if 'y\n' is not found
    print("Found 'y\\n' in the string.")

Found 'n\n' in the string!


In [ ]:
    chat_session = model.start_chat(
    history=[  ])

    response = chat_session.send_message(prompt)
    #print(response.text)
    return response.text
prompt = "What is today's date?"
message_gpt(prompt)

## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [18]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [19]:
get_ticket_price("Berlin")
#get_ticket_price("Texas")

Tool get_ticket_price called for Berlin


'$499'

In [20]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [22]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [ ]:
# # Provided code: Not using as uses open AI
# def chat(message, history):
#     messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
#     response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

#     if response.choices[0].finish_reason=="tool_calls":
#         message = response.choices[0].message
#         response, city = handle_tool_call(message)
#         messages.append(message)
#         messages.append(response)
#         response = openai.chat.completions.create(model=MODEL, messages=messages)
    
#     return response.choices[0].message.content

# # We have to write that function handle_tool_call:

# def handle_tool_call(message):
#     tool_call = message.tool_calls[0]
#     arguments = json.loads(tool_call.function.arguments)
#     city = arguments.get('destination_city')
#     price = get_ticket_price(city)
#     response = {
#         "role": "tool",
#         "content": json.dumps({"destination_city": city,"price": price}),
#         "tool_call_id": tool_call.id
#     }
#     return response, city

# gr.ChatInterface(fn=chat, type="messages").launch()

In [23]:
# Provided code: Not using as uses open AI
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    #response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    response = ask_question(prompt, history, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        #response = openai.chat.completions.create(model=MODEL, messages=messages)
        response = ask_question(prompt, history)
    
    return response.choices[0].message.content

# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city


#gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\Tim_S\Desktop\bt\AIEng\llm_engineering\.venv\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Tim_S\Desktop\bt\AIEng\llm_engineering\.venv\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Tim_S\Desktop\bt\AIEng\llm_engineering\.venv\Lib\site-packages\gradio\blocks.py", line 2047, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Tim_S\Desktop\bt\AIEng\llm_engineering\.venv\Lib\site-packages\gradio\blocks.py", line 1592, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Tim_S\Desktop\bt\AIEng\llm_engineering\.venv\Lib\site-packages\grad